In [ ]:
import os, sys
from keras import backend as K

%matplotlib inline

In [ ]:
homeDir = os.getcwd()
dataDir = homeDir + "/data/"
#dataDir = homeDir + "/data/sample/"
train_path = dataDir + "train/"
valid_path = dataDir + "valid/"
model_path = homeDir + "/data/models/"
test_path = dataDir + "test"
results_path = dataDir + "/results"

In [ ]:
from utils import *
from Vgg16 import Vgg16
from vgg16bn import Vgg16BN

In [ ]:
vgg = Vgg16BN()

In [ ]:
#Set constants. You can experiment with no_of_epochs to improve the model. You can reduce the batch_size 
#depending on the memory contraints of gpu
batch_size=64

In [ ]:
# get_batches function transforms images into arrays and the gets them in batches.
batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size)

In [ ]:
vgg.model.pop() #Remove the final layer
for layer in vgg.model.layers:
    layer.trainable=False # Set all other layers to untrainable

In [ ]:
vgg.model.add(Dense(2, activation='softmax')) 
#Adding a new dense layer wiht only 2 outputs and softmax acitvation as it is the output layer

In [ ]:
vgg.compile() #using the default compiler
#vgg.model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
vgg.fit(batches, val_batches, nb_epoch=1)

In [ ]:
model = vgg.model
layers = model.layers
lastConvIdx = [index for index,layer in enumerate(layers) if type(layer) is Convolution2D][-1]
convLayers = layers[:lastConvIdx+1]
convModel = Sequential(convLayers)
fcLayers = layers[lastConvIdx+1:]


In [ ]:
batches = get_batches(train_path, shuffle=False, batch_size = batch_size)
val_batches = get_batches(valid_path, shuffle=False, batch_size=batch_size)
val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

In [ ]:
train_features = convModel.predict_generator(batches,batches.nb_sample)
val_features = convModel.predict_generator(val_batches, val_batches.nb_sample)

In [ ]:
save_array(model_path + 'train_convlayer_features.bc', train_features)
save_array(model_path + 'valid_convlayer_features.bc', val_features)

In [ ]:
trn_features = load_array(model_path+'train_convlayer_features.bc')
val_features = load_array(model_path+'valid_convlayer_features.bc')

In [ ]:
def new_weights(layer, prev_p, new_p):
    scal = (1-prev_p)/(1-new_p)
    return [o*scal for o in layer.get_weights()]
opt = RMSprop(lr=0.00001, rho=0.7)
def get_fc_model(p):
    model = Sequential([
        MaxPooling2D(input_shape = convLayers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096,activation="relu"),
        BatchNormalization(),
        Dropout(p),        
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(2, activation='softmax')
    ])
    for l1,l2 in zip(model.layers, fcLayers): l1.set_weights(new_weights(l2,0.5,p))

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
fc_model = get_fc_model(0.6)

In [ ]:
fc_model.fit(trn_features, trn_labels, nb_epoch=5, 
             batch_size=batch_size, validation_data=(val_features, val_labels))

### Psuedo labelling

In [ ]:
test_path = dataDir + "/test"
test_batches = get_batches(test_path,shuffle=False, batch_size = batch_size)

In [ ]:
#get the test set features first
test_features = convModel.predict_generator(test_path,shuffle=False,batch_size = batch_size)

In [ ]:
test_labels = full_model.predict_generator(test_batches,nb.sample)

In [ ]:
save_array(results_path + 'test_features.dat', test_features)

In [ ]:
test_features = load_array(results_path + 'test_features.dat')

In [ ]:
combined_features = np.concantanate([trn_features,test_features])
combined_labels = np.concantanate([trn_labels,test_labels])

In [ ]:
fc_model.fit(combined_features, combined_labels, nb_epoch=1, 
             batch_size=batch_size, validation_data=(val_features, val_labels))

## Predictions

In [ ]:
test_path = dataDir + "/test"
test_batches = get_batches(test_path,shuffle=False, batch_size = batch_size)

In [ ]:
preds = convModel.predict_generator(test_batches, test_batches.nb_sample)

In [ ]:
filenames = test_batches.filenames
isdog = preds[:,1]
isdog = isdog.clip(min=0.025, max=0.975)
ids = np.array([int(f[8:f.find('.')]) for f in filenames])
subm = np.stack([ids,isdog], axis=1)
subm[:5]
%cd $dataDir
submission_file_name = 'submission_ft_denseLayers_droupout6.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')
from IPython.display import FileLink
%cd ../
FileLink('data/'+submission_file_name)